In [530]:
class Stockdb
  require 'csv'
  attr_accessor :table
  attr_accessor :headers
  attr_accessor :path
  attr_accessor :stockpath
  attr_accessor :tsvtable
  
  def initialize (stockpath:)
    unless File.file?(stockpath)
      return ("File doesn't exist")
    else
      @path = Dir.pwd # Current working directory
      @stockpath = stockpath # Seed_stock_data.tsv path
      @table = CSV.read(File.open(stockpath), headers: true, col_sep: "\t") # Importing stockfile as table
      @headers = table.headers
      @tsvtable = CSV.parse(File.open(stockpath), headers: true) #tab separated table for data inspection
      
      table.headers.each do |header|
        define_singleton_method "get_#{header.downcase}" do |match| # Dinamycally declaring methods for each header 
            col = @table[header] 
            index = col.each_index.select{ |x| col[x] == match} #Selecting column indexes that match input
            puts @headers.join("\t") #printing headers to preserve table info
            index.each { |i| row = "#{@table[i]}" 
            row = row.gsub(",", "\t") # Selecting index's row and replacing , for tab
            puts row
            }
            return 
        end
      end
    end
  end

  def Stockdb.new_database (stockpath, newdb)
    @path = Dir.pwd
    @table = CSV.read(File.open(stockpath), headers: true, col_sep: "\t")
    newstock = @table
    newstock.each { |row| #Iterating over each row
      row[4] = (row[4].to_i - 7) # Substraction from Grams_Remaining
      if row[4] <= 0 # return error message if value equal or < 0
        row[4]=0
        $stderr.puts "WARNING: we have run out of Seed Stock #{row[0]}"
      end
    }
    CSV.open("#{@path}/#{newdb}", 'w', col_sep: "\t") do |tsv| #Creating file to save new data in pwd
      tsv << newstock.headers #saving headers
      newstock.each { |row| tsv << row } #saving modified rows
    end
    return newstock
  end
  
  #https://www.toptal.com/ruby/ruby-metaprogramming-cooler-than-it-sounds

end

#Testing 
prueba2 = Stockdb.new(stockpath: "/home/osboxes/bioinfogit/bioinfo_Chals/seed_stock_data.tsv")
prueba = Stockdb.new_database("/home/osboxes/bioinfogit/bioinfo_Chals/seed_stock_data.tsv", 'newstock.tsv')
#prueba2.get_Seed_Stock('A348')
#stock.get_Mutant_Gene_ID('AT5G20240')
puts stock.get_Seed_Stock('A51')
#puts stock.new_database('newstock.tsv')
#stock.headers

Seed_Stock	Mutant_Gene_ID	Last_Planted	Storage	Grams_Remaining
A51	AT1G30950	9/2/2013	cama25	5



In [520]:
clases[0].new_stockdb('a')

NoMethodError: undefined method `each' for nil:NilClass

In [ ]:
  
  attr_accessor :seed_stock
  attr_accessor :mutant_gene_id
  attr_accessor :last_planted
  attr_accessor :storage
  attr_accessor :grams_remaining
  attr_accessor :objects


      @seed_stock = seed_stock
      @mutant_gene_id = mutant_gene_id
      @last_planted = last_planted
      @storage = storage
      @grams_remaining = grams_remaining  

  def Stockdb.load_stock (stockpath:)
    unless File.file?(stockpath)                # Check if file exists
      return "This file path does not exist"
    else
      stock_array = CSV.open(stockpath, col_sep: "\t", headers:true).read
      stock_object_list = Array.new  ## Empty array created to append gene objects.

      # Conversion of genes into Gene objects
      stock_array.each do |stock|
        stock_object = Stockdb.new(seed_stock: stock[0],
          mutant_gene_id: stock[1],last_planted: stock[2], 
          storage: stock[3], grams_remaining: stock[4])
        stock_object_list << stock_object
      end
      @objects = stock_object_list
      return stock_object_list
    end
  end